In [1]:
!pip install kaggle

In [2]:
# Step 1: Install Kaggle Library
!pip install kaggle

In [3]:
# Step 3: Configure Kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
  0% 0.00/80.9M [00:00<?, ?B/s]
100% 80.9M/80.9M [00:00<00:00, 1.38GB/s]


In [5]:
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset,'r') as zip:
    zip.extractall()
    print('The dataset is extracted')

The dataset is extracted


In [6]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [21]:
import nltk
nltk.download('stopwords')
stopwords=stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [23]:
print(stopwords)

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [9]:
column_names=['target','id','date','flag','user','text']

In [52]:
df=pd.read_csv('/content/sentiment140.zip',names=column_names,encoding="ISO-8859-1")

In [53]:
df.head(10)

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
5,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
6,0,1467811592,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,mybirch,Need a hug
7,0,1467811594,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,coZZ,@LOLTrish hey long time no see! Yes.. Rains a...
8,0,1467811795,Mon Apr 06 22:20:05 PDT 2009,NO_QUERY,2Hood4Hollywood,@Tatiana_K nope they didn't have it
9,0,1467812025,Mon Apr 06 22:20:09 PDT 2009,NO_QUERY,mimismo,@twittera que me muera ?


In [54]:
df.shape

(1600000, 6)

In [55]:
df.isnull().sum()

,0
target,0
id,0
date,0
flag,0
user,0
text,0


In [56]:
df['target'].value_counts()

,count
target,
0,800000
4,800000


In [57]:
df.replace({'target':{4:1}},inplace=True)

In [58]:

df['target'].value_counts()

,count
target,
0,800000
1,800000


In [59]:
#0->Negative tweet
#1->Positive tweet
#Stemming is to reduce the word to it's root word
port_stem=PorterStemmer()

In [60]:
import re
from nltk.corpus import stopwords

# def clean_tweet(text):
#     text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
#     text = re.sub(r'\@\w+|\#','', text)
#     text = re.sub(r"[^a-zA-Z']", ' ', text)
#     text = text.lower()
#     text = ' '.join(word for word in text.split() if word not in stopwords.words('english'))
#     return text
# def stemming(content):
#   #remove any character that's not alphabet
#   stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
#   #make it all lower letters
#   stemmed_content=stemmed_content.lower()

#   stemmed_content=stemmed_content.split()
#   #stem all words after aplitting but not the  stop words
#   stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
#   # to join the words again
#   stemmed_content=' '.join(stemmed_content)
#   return stemmed_content
# df['cleaned_content'] = df['text'].apply(clean_tweet)

# from nltk.stem import WordNetLemmatizer
# lemmatizer = WordNetLemmatizer()

# def lemmatize_text(text):
#     return



In [61]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')  # Also needed for some lemmatization tasks


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [62]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download once if needed
# import nltk
# nltk.download('stopwords')
# nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_tweet(text):
    # Remove URLs
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    # Remove mentions and hashtags
    text = re.sub(r'\@\w+|\#','', text)
    # Remove non-letter characters
    text = re.sub(r"[^a-zA-Z']", ' ', text)
    # Lowercase
    text = text.lower()
    # Tokenize, remove stopwords, and lemmatize
    tokens = text.split()
    cleaned_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return ' '.join(cleaned_tokens)


In [63]:
df['cleaned_content']=df['text'].apply(preprocess_tweet)

In [64]:
# df['stemmed_content']=df['text'].apply(stemming)

In [65]:
df.head()

,target,id,date,flag,user,text,cleaned_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",that's bummer shoulda got david carr third day
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset can't update facebook texting might cry ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,dived many time ball managed save rest go bound
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole body feel itchy like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",behaving mad can't see


In [66]:
# df['stemmed_content']

In [67]:
X=df['cleaned_content'].values
Y=df['target'].values

In [68]:
X

array(["that's bummer shoulda got david carr third day",
       "upset can't update facebook texting might cry result school today also blah",
       'dived many time ball managed save rest go bound', ...,
       'ready mojo makeover ask detail',
       'happy th birthday boo alll time tupac amaru shakur',
       'happy charitytuesday'], dtype=object)

In [69]:
Y

array([0, 0, 0, ..., 1, 1, 1])

In [70]:
#stratify to have equal propotional for my training data to be the same in train and test same distrubution for the two classses
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

In [71]:
print(X.shape,X_train.shape,X_test.shape)

(1600000,) (1280000,) (320000,)


In [72]:
print(X_train)

['watch saw iv drink lil wine' ''
 'even though favourite drink think vodka coke wipe mind time think im gonna find new drink'
 ... 'eager monday afternoon'
 "hope everyone mother great day can't wait hear guy store tomorrow"
 'love waking folgers bad voice deeper']


In [73]:
print(X_test)

['fine much time chat twitter hubby back summer amp tends dominate free time'
 'ahs may show w ruth kim amp geoffrey sanhueza'
 'maybe bay area thang dammit' ...
 'nevertheless hooray member wonderful safe trip' 'feeling well' 'thank']


In [74]:
#In the training data the vectorizer will give score to all the individual words ,
#fitting means to understand what is the nature of the data is present in the training data ,
#transform means converting the data to numerical values
#in X_test we just use transform because test data has to be tranformed based on how the training data transformed
#fit(): Learns the vocabulary and IDF (inverse document frequency) from the training data.
#transform(): Applies that learned vocabulary to convert the text into a vectorized form which is numerical vector
#X_test = vectorizer.transform(X_test) ,This only transforms the test data using the vocabulary and IDF that were learned from the training data.
'''
Why not fit_transform() on the test data?
Because:

The model should never learn anything from the test set. That would be data leakage.
The test set must be transformed using the same vocabulary and IDF as the training set
, otherwise the model is seeing different representations during training and testing'''


'\nWhy not fit_transform() on the test data?\nBecause:\n\nThe model should never learn anything from the test set. That would be data leakage.\nThe test set must be transformed using the same vocabulary and IDF as the training set\n, otherwise the model is seeing different representations during training and testing'

In [75]:
#converting into numercial data , just fit the training data but dont fit the test we just tranform the test data
vectorizer = TfidfVectorizer(ngram_range=(1,2), max_df=0.9, min_df=5, max_features=5000)
X_train=vectorizer.fit_transform(X_train)
X_test=vectorizer.transform(X_test)

In [76]:
print(X_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 8031680 stored elements and shape (1280000, 5000)>
  Coords	Values
  (0, 4708)	0.3214539478382776
  (0, 3686)	0.34777136271025044
  (0, 2216)	0.5158795484963832
  (0, 1144)	0.39035169889354276
  (0, 2498)	0.40938748563048066
  (0, 4812)	0.43545168264812906
  (2, 1144)	0.45438083562067577
  (2, 1260)	0.18026842750704633
  (2, 4332)	0.1725556773201698
  (2, 1360)	0.26848385853148776
  (2, 4310)	0.30617947619818986
  (2, 4636)	0.30403491387431797
  (2, 754)	0.28898340823233154
  (2, 2777)	0.2235670539058346
  (2, 4363)	0.14021492720695863
  (2, 2122)	0.15052735677311505
  (2, 1697)	0.17342569249501213
  (2, 1423)	0.19012326330712526
  (2, 2949)	0.15485944687155734
  (2, 1265)	0.2577755210102959
  (2, 4317)	0.275283114371843
  (2, 2131)	0.2544662458553404
  (3, 4310)	0.2978753564607449
  (3, 1877)	0.44731992447997165
  (3, 1758)	0.27683184284080126
  :	:
  (1279996, 4151)	0.29824028720230933
  (1279996, 4423)	0.4465132744151447


In [77]:
print(X_test)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2006524 stored elements and shape (320000, 5000)>
  Coords	Values
  (0, 122)	0.21151909140589265
  (0, 264)	0.20001000878900346
  (0, 676)	0.3420328264558254
  (0, 1429)	0.3143962196895383
  (0, 1508)	0.2915225940000334
  (0, 2081)	0.34572253660061114
  (0, 2860)	0.21915460864277467
  (0, 2865)	0.41692542388848225
  (0, 4129)	0.2704356428112285
  (0, 4363)	0.38725350906724043
  (0, 4524)	0.223513912629559
  (1, 122)	0.35118510502987177
  (1, 2317)	0.6885850836811859
  (1, 2711)	0.47874553334780373
  (1, 3815)	0.4163199717002046
  (2, 188)	0.5168245273761574
  (2, 319)	0.5557541082403169
  (2, 926)	0.5316479012521239
  (2, 2713)	0.37600570231132613
  (3, 1224)	0.6835012983147498
  (3, 1556)	0.5156283558028295
  (3, 2590)	0.5166753079973165
  (4, 511)	0.755162692861146
  (4, 823)	0.6555374186960667
  (5, 4776)	0.5276913837108701
  :	:
  (319994, 4720)	0.24282670407172827
  (319994, 4835)	0.30079309864144055
  (319995, 1126)	0.

In [78]:
#max_iter shich means the maximmum number of times the model can go through the data
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.01, 0.1, 1, 10]}
grid = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid.fit(X_train, y_train)


GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [0.01, 0.1, 1, 10]})

In [79]:
# model.fit(X_train,y_train)

NameError: name 'model' is not defined

In [80]:
#Model evaluation on training data
train_predict=grid.predict(X_train)
accuracy=accuracy_score(y_train,train_predict)
print(accuracy)

0.77624140625


In [81]:
#Evaluate on test_set
test_predict=grid.predict(X_test)
accuracy=accuracy_score(y_test,test_predict)
print(accuracy)

0.7744625


In [84]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.svm import LinearSVC
clf = LinearSVC()
clf.fit(X_train, y_train)


LinearSVC()

In [85]:
y_pred = clf.predict(X_test)


In [86]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.77364375
Confusion Matrix:
 [[119095  40905]
 [ 31529 128471]]
Classification Report:
               precision    recall  f1-score   support

           0       0.79      0.74      0.77    160000
           1       0.76      0.80      0.78    160000

    accuracy                           0.77    320000
   macro avg       0.77      0.77      0.77    320000
weighted avg       0.77      0.77      0.77    320000

